# Generate DFS

Este notebook tem como função gerar os dataframes que serão utilizados no desenvolvimento do projeto, para tanto, teremos três data frames principais:
* Dados transacionais: dados de cada usuário em um range de tempo específico, ou seja, quais músicas e genero aquele usuário escutou (optei por ultilizar os primeiros seis meses, delineando o estudo como sazonal semestral)
* Dados informacionais do genero das músicas
* Dados dos artistas

### Requirements

In [2]:
import random
import numpy as np
import pandas as pd
from datetime import datetime

### Functions

## transactional data

In [3]:
def create_df_modeling(start_date, end_date,
                       low_user, high_user, dau,
                       low_track, high_track, 
                       min_plays, max_plays,
                       min_n_tracks, max_n_tracks
                      ):
    # creates a range of dates
    date_range = pd.date_range(start_date, end_date)
    # creates empty array for the final user array
    user_array_acum = []
    # creates empty array for the final id_date array
    date_array_acum = []
    # creates empty array for the final track array
    tracks_array_acum = []
    # creates empty array for the final plays array
    plays_array_acum = []
    # minimum number of random listeners per day (dau)
    length_user = 1 + int((high_user-low_user) * dau)
    # for loop to random create transactional data
    for current_date in date_range:
        #
        user_array = np.random.randint(low_user, high_user, size = length_user)
        user_array_col = np.reshape(user_array, (len(user_array), 1))
        user_array_acum.append(user_array_col)
        #
        date_array = current_date.strftime("%Y-%m-%d")
        date_array = np.full(len(user_array), current_date)
        date_array_col = np.reshape(date_array, (len(date_array), 1))
        date_array_acum.append(date_array_col)
    # for loop that ranges from the min id_user to the max id_user
    for k in range(0,len(np.concatenate(user_array_acum))):
        # defines randomly the maximum range of tracks a listener can listen in 1 single day
        length_size = random.randint(min_n_tracks, max_n_tracks)
        # selects at random an array of id_track for the specific listener
        tracks_array = np.random.randint(low_track, high_track, size = length_size)
        tracks_array_acum.append(tracks_array)
        # select at random the average of plays given by each user on a giving day
        plays_array = np.random.randint(min_plays, max_plays, size = length_size)
        plays_array_acum.append(plays_array) 
    # concatenates the arrays of date, id_user and plays on the final dataframe
    date_array_final = np.concatenate(date_array_acum, axis = 0)
    user_array_final = np.concatenate(user_array_acum, axis = 0)
    # creates the final transactional dataframe with id_users and number of plays for each day
    df = pd.DataFrame({'id_date':date_array_final.reshape(len(date_array_final)),
                       'user_id':user_array_final.reshape(len(user_array_final)),
                       'id_tracks': tracks_array_acum,
                       'plays':plays_array_acum
                      })
    # explodes the final dataframe to create a vertical transactional table
    df = df.explode(['id_tracks','plays']).reset_index(drop=True)
    #
    return df

## Dimensional content

In [4]:
def create_dim_content(n_tracks, min_tracks, max_tracks, n_artists, n_genres):
    # creates an initial list of id_tracks
    id_track_list = [i for i in range(1,n_tracks+1)]
    # creates an initial list of id_artists
    id_artist_list = [i for i in range(1, n_artists+1)]
    # creates an initial list of id_genres
    id_genres = [i for i in range(1,n_genres)]
    # creates array with the specific size of tracks by artists to be distributed amongst all artists
    artist_array_sizes = np.random.randint(min_tracks, max_tracks, size = n_artists).tolist()
    # Create a copy of the list
    list_copy = id_track_list.copy()
    # initiates an empty array to store results of for loop append (arrays of id_tracks)
    sampled_arrays_acum = []
    # initiates an empty array to store results of for loop append (arrays of id_genre)
    sampled_arrays_genre_acum = []
    # for loop to interate and create the dimensional data for id_artists and id_tracks
    for size in artist_array_sizes:
        if size <= len(list_copy):
            sampled_arrays = random.sample(list_copy, size)
            sampled_arrays_genre = random.choices(id_genres, k = size)
            sampled_arrays_acum.append(sampled_arrays)
            sampled_arrays_genre_acum.append(sampled_arrays_genre)
            list_copy = [item for item in list_copy if item not in sampled_arrays]
    # creates the final dimensional dataframe
    df = pd.DataFrame({'id_artist':id_artist_list,
                       'id_tracks':sampled_arrays_acum,
                       'id_genre':sampled_arrays_genre_acum
                      })
    # explodes the final dataframe to create a vertical dimensional table
    df = df.explode(['id_tracks','id_genre']).reset_index(drop = True)
    #
    return df

## Features for artists

In [5]:
def creates_features(n_features,n_artists,std_max, min_mean, max_mean):
    # creates an array with standard deviation of a uniform distribution
    std_array = [random.uniform(0, std_max) for i in range(0,n_features)]
    # creates an array with mean values of a uniform distribution
    mean_array = [random.randint(min_mean, max_mean) for i in range(0,n_features)]
    # creates an empty array with the number of rows as the same of the length of the quantity of artists
    result = np.empty((n_artists, 0))
    for i in range(0,n_features):
        feature_i = np.random.normal(mean_array[i], std_array[i] * mean_array[i], n_artists)
        feature_i = feature_i.astype(int)
        result = np.column_stack((result, feature_i))
        result = result.astype(int)
    # creates the list of id_artist
    id_artist = [i for i in range(1,n_artists+1)]
    # fills values on the result array
    result = np.column_stack((id_artist,result))
    # creates a list with only one element
    artist_col_names = ['id_artist']
    # creates a list with the names of the features that will be used on the final dataframe
    feat_col_names = ['Feature'+str(i) for i in range(1,n_features+1)]
    # adds two lists (elements will be use as the header of the final dataframe)
    col_names = artist_col_names + feat_col_names
    # final dataframe with features by artist
    df = pd.DataFrame(result, columns = col_names)
    return df

# Pipeline

## Defining global parameters

In [6]:
########################################################################################################
######################### input parameters for the create_df_modeling function #########################
# Define the range of dates for the trasactional dataset
start_date = datetime(2021, 7, 30).strftime('%Y-%m-%d')
end_date = datetime(2023, 7, 30).strftime('%Y-%m-%d')
# defines the minimum id_user to consider
low_user = 1
# defines the maximum id_user to consider
high_user = 50
# defines the daily audience (the percentage of the sample size of users) for each id_date
dau = 0.1
# defines the minimum id_track available
low_track = 1
# defines the maximum id_track available
high_track = 4000
# defines the minimum number of plays a listener can hit play every day
min_plays = 1
# defines the maximum number of plays a listener can hit play every day
max_plays = 20
# defines the minimum number of tracks a listener can listen to every day
min_n_tracks = 2
# defines the maximum number of tracks a listener can listen to every day
max_n_tracks = 5


########################################################################################################
################ input parameters for the create_dim_content function ##################################
# defines the total number of tracks
n_tracks = high_track
# defines the total number of id_artist
n_artists = 500
# defines the total number of genres
n_genres = 5
# defines the minimum number of tracks by id_artist
min_tracks = 2 * round((n_tracks/n_artists),0) * 0.2
# defines the maximum number of tracks by id_artist
max_tracks = 2 * round((n_tracks/n_artists),0) * 0.8


########################################################################################################
################ input parameters for the creates_features function ##################################
# number of features that are going to appear on the final dataframe
n_features = 10
# maximum standard deviation value for each of the feature
std_max = 1
# minimum value for the mean value for each of the features
min_mean = 100
# maximum value for the mean value for each of the features
max_mean = 1500

## Creates df transactional data

In [7]:
#
df = create_df_modeling(start_date, end_date,
                        low_user, high_user, dau,
                        low_track, high_track, 
                        min_plays, max_plays,
                        min_n_tracks, max_n_tracks
                       )

## Creates df dimensional content

In [8]:
dc = create_dim_content(n_tracks,min_tracks,max_tracks,n_artists, n_genres)

## Creates df with features by artist

In [9]:
df_feat = creates_features(n_features,n_artists,std_max, min_mean, max_mean)

# Artefacts

## Transactional data source (reference modeling dataframe)

In [10]:
df

,id_date,user_id,id_tracks,plays
0,2021-07-30,9,182,18
1,2021-07-30,9,3663,14
2,2021-07-30,9,3268,9
3,2021-07-30,4,178,3
4,2021-07-30,4,379,1
...,...,...,...,...
12771,2023-07-30,23,3193,5
12772,2023-07-30,23,115,15
12773,2023-07-30,18,1661,1
12774,2023-07-30,18,1965,19


In [11]:
df.dtypes

id_date      datetime64[ns]
user_id               int32
id_tracks            object
plays                object
dtype: object

In [12]:
df.isnull().sum()

id_date      0
user_id      0
id_tracks    0
plays        0
dtype: int64

In [13]:
# identificando feriados
from datetime import datetime, date
import pandas as pd
from pandas.tseries.holiday import AbstractHolidayCalendar, GoodFriday, Holiday, Easter, Day

class Feriados(AbstractHolidayCalendar):
    rules = [
        Holiday('Confraternização Universal', month=1, day=1),
        Holiday('Aniversário de São Paulo', month=1, day=25),
        Holiday('Segunda-Feira de Carnaval', month=1, day=1, offset=[Easter(), Day(-48)]),
        Holiday('Terça-Feira de Carnaval', month=1, day=1, offset=[Easter(), Day(-47)]),
        Holiday('Quarta-Feira de Cinzas', month=1, day=1, offset=[Easter(), Day(-46)]),
        # Sexta-feira Santa
        GoodFriday,
        Holiday('Corpus Christi', month=1, day=1, offset=[Easter(), Day(60)]),
        Holiday('Tiradentes', month = 4, day = 21),
        Holiday('Dia do Trabalho', month = 5, day = 1),
        Holiday('Revolução Constitucionalista', month=7, day=9, start_date='1997-01-01'),
        Holiday('Independência do Brasil', month = 9, day = 7),
        Holiday('Nossa Senhora Aparecida', month = 10, day = 12),
        Holiday('Finados', month = 11, day = 2),
        Holiday('Proclamação da República', month = 11, day = 15),
        Holiday('Dia da Consciencia Negra', month=11, day=20, start_date='2004-01-01'),
        Holiday('Vespera de Natal', month=12, day=24),
        Holiday('Natal', month = 12, day = 25)]

sp_cal = Feriados()
sp_feriados = pd.offsets.CustomBusinessDay(calendar=sp_cal)
feriados_sp = sp_cal.holidays(datetime(2000, 12, 31), datetime(2079, 12, 31))

start = datetime(2021, 7, 30)
end = datetime(2023, 7, 30)
date_fds = pd.bdate_range(start, end, freq='C', holidays=feriados_sp)

In [17]:
feriados_sp

DatetimeIndex(['2001-01-01', '2001-01-25', '2001-02-26', '2001-02-27',
               '2001-02-28', '2001-04-13', '2001-04-21', '2001-05-01',
               '2001-06-14', '2001-07-09',
               ...
               '2079-05-01', '2079-06-22', '2079-07-09', '2079-09-07',
               '2079-10-12', '2079-11-02', '2079-11-15', '2079-11-20',
               '2079-12-24', '2079-12-25'],
              dtype='datetime64[ns]', length=1340, freq=None)

In [23]:
df['holiday'] = np.where(df.id_date.isin(feriados_sp), 0, 1)
df['plays'] = df.plays.astype(int)

In [25]:
df.describe()

,id_date,user_id,plays,holiday
count,12776,12776.000000,12776.000000,12776.000000
mean,2022-07-30 08:48:23.443957504,24.821697,9.924233,0.953976
min,2021-07-30 00:00:00,1.000000,1.000000,0.000000
25%,2022-01-27 00:00:00,13.000000,5.000000,1.000000
50%,2022-08-01 00:00:00,25.000000,10.000000,1.000000
75%,2023-01-29 00:00:00,37.000000,15.000000,1.000000
max,2023-07-30 00:00:00,49.000000,19.000000,1.000000
std,NaN,14.133237,5.510116,0.209545


In [26]:
df.user_id.nunique()

49

## Dimensional data source with musical genre for each track

In [27]:
dc

,id_artist,id_tracks,id_genre
0,1,1101,4
1,1,3240,3
2,1,3422,3
3,1,1704,4
4,1,970,1
...,...,...,...
3550,500,3853,2
3551,500,2132,3
3552,500,2631,4
3553,500,3098,4


In [28]:
dc.dtypes

id_artist     int64
id_tracks    object
id_genre     object
dtype: object

In [29]:
dc.isnull().sum()

id_artist    0
id_tracks    0
id_genre     0
dtype: int64

In [30]:
dc.id_artist.nunique()

500

In [31]:
dc.id_tracks.nunique()

3555

In [32]:
dc.id_genre.nunique()

4

## Data source with feature values for each artist

In [33]:
df_feat

,id_artist,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Feature10
0,1,144,449,542,565,1257,249,230,334,-147,618
1,2,133,799,511,573,1208,2954,229,985,3292,595
2,3,108,681,388,533,1397,2943,241,1794,3304,1075
3,4,71,1035,691,642,1422,560,245,789,1109,1220
4,5,220,457,335,579,1322,1758,238,1718,862,-67
...,...,...,...,...,...,...,...,...,...,...,...
495,496,140,499,459,496,1194,1451,239,1623,-1321,838
496,497,38,733,774,555,1726,1582,235,1515,2008,613
497,498,57,452,621,589,1269,1693,220,2344,1037,2339
498,499,119,588,914,651,329,-522,236,772,1883,1230


In [34]:
df_feat.dtypes

id_artist    int32
Feature1     int32
Feature2     int32
Feature3     int32
Feature4     int32
Feature5     int32
Feature6     int32
Feature7     int32
Feature8     int32
Feature9     int32
Feature10    int32
dtype: object

In [35]:
df_feat.isnull().sum()

id_artist    0
Feature1     0
Feature2     0
Feature3     0
Feature4     0
Feature5     0
Feature6     0
Feature7     0
Feature8     0
Feature9     0
Feature10    0
dtype: int64

## Juntando as bases

In [36]:
dc

,id_artist,id_tracks,id_genre
0,1,1101,4
1,1,3240,3
2,1,3422,3
3,1,1704,4
4,1,970,1
...,...,...,...
3550,500,3853,2
3551,500,2132,3
3552,500,2631,4
3553,500,3098,4


In [37]:
df_all = df.merge(dc, how='left').merge(df_feat, how = 'left')

In [38]:
df_all.to_parquet(r'data/raw_recommendation_data.parquet')

## Conclusões

Sobre as features:
artistas unicos: 500

user unicos: 49

tracks unicas: 3495

Temos valores nulos entre a base de df e dc

Sobre o delineamento do problema: Ultilizei um periodo de sazonalidade semestral neste primeiro momento, porém como proximas evoluções do modelo, seria interessante desenvolver um modelo de recomendação para cada época do ano, entendendo que a temperatura média e a estação do ano pode influenciar na nmúsica a ser ouvida. Uma analogia são músicas de natal no proprio natal.
Além disso, seria interessante também explorar um modelo para dias uteis e não uteis, com a hipotese de que em dias uteis ouvimos mais músicas de foco para trabalhar e dias não uteis músicas de descansar.

Com isso, juntei as bases e passamos a proxima etapa, resolução de dados faltantes e festure engineering das variáveis.